In [70]:
import json
import numpy
import requests
import xml.etree.ElementTree as ET

import warnings
warnings.filterwarnings('ignore')

In [73]:
def getElev(lat, lon, buffer):
    """
    Function to get the DEM data for a given latitude and longitude.
    Latitude and Longitude should be in WGS83 
    
    # EPQS
    # https://nationalmap.gov/epqs/pqs.php
    # x=-122
    # y=44
    # units=Meters
    # output=json


    """
    
    root = 'https://nationalmap.gov/epqs/pqs.php'

    transforms = [[-1, 1],  [0, 1],   [1, 1],
                  [-1, 0],  [0, 0],  [1, 0],
                  [-1, -1], [0, -1], [1, -1]]
    elev = []
    x = []
    y = []

    for i in range(0, 9):
        qlon = lon  + buffer*transforms[i][0]
        qlat = lat + buffer*transforms[i][1]

        # assumes that the input coords are specific in latitude/longitude (NAD 1983)
        params = {'x': qlon,
                  'y': qlat,
                  'units':'Meters',
                  'output':'json'}

        # make the WFS request
        r = requests.get(url = root, params = params, verify=False)


        # parse the respone
        res = json.loads(r.text)
        dat = res['USGS_Elevation_Point_Query_Service']['Elevation_Query']
        elev.append(dat['Elevation'])
        x.append(dat['x'])
        y.append(dat['y'])

    elev = numpy.array(elev).reshape(-1,3)
    x = numpy.array(x).reshape(-1,3)
    y = numpy.array(y).reshape(-1,3)

    return elev, x, y

In [74]:
lat = 44.8004
lon = -122.6089
buffer = 0.01

elev, x, y = getElev(lat, lon, buffer)

In [75]:
elev

array([[428.45, 398.37, 484.87],
       [297.6 , 312.91, 296.87],
       [193.6 , 205.37, 203.39]])

In [ ]:
# todo:
# convert to rasterio object?
# compute slope, aspect, etc.

In [15]:
import rasterio
from rasterio.transform import from_origin

In [18]:
r.raw.decode_content = True

In [ ]:
# WEB COVERAGE SERVICE - CANNOT GET THIS TO WORK
# 'https://elevation.nationalmap.gov/arcgis/services/3DEPElevation/ImageServer/WCSServer'
# request=GetCoverage
# service=WCS
# version=1.0.0
# Coverage=" + str(c.id) + "
# bbox=" + str(rBBOX) + "
# format=" +  str(rfmt) + "
# width=" + str(resx) + "
# height=" + str(resy) + "
# crs=" + str(rCRS) + str(kvp_sep)

# # the url of the web service for WBD info
# root = 'https://elevation.nationalmap.gov/arcgis/services/3DEPElevation/ImageServer/WCSServer'

# https://elevation.nationalmap.gov:443/arcgis/services/3DEPElevation/ImageServer/WCSServer?
#SERVICE=WCS
#REQUEST=GetCoverage
#VERSION=2.0.1
#COVERAGEID=DEP3Elevation
#SUBSET=x(-20037507.0672000013,-11911185.14836644)
#SUBSET=y(4474012.1150791775,4478208.115037268)
#SCALESIZE=x(33),y(42)
#Format=image/tiff


{'huc12': '170900050601',
 'huc10': '1709000506',
 'huc8': '17090005',
 'huc6': '170900',
 'huc4': '1709',
 'huc2': '17'}